In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, ConfusionMatrixDisplay, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, RocCurveDisplay
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from datetime import datetime
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

In [23]:
filename = "C:/Users/rodlu/OneDrive/Escritorio/Coding_Dojo/Data_Science/Data sets/credit_customers.csv"
df = pd.read_csv(filename)
df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,...,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes,good
1,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,female div/dep/mar,none,...,real estate,22.0,none,own,1.0,skilled,1.0,none,yes,bad
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,male single,none,...,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,yes,good
3,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,male single,guarantor,...,life insurance,45.0,none,for free,1.0,skilled,2.0,none,yes,good
4,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,male single,none,...,no known property,53.0,none,for free,2.0,skilled,2.0,none,yes,bad


In [24]:
# Crear una copia del dataframe original
df_1 = df.copy()

Ingenieria de Caracteristicas

In [25]:
# eliminar la columna 'checking_status'
df_1.drop('checking_status', axis=1, inplace=True)
# Reemplazar valores inconsistentes en la columna 'credit_history': 'no credits/all paid' por 'no credits'
df_1['credit_history'] = df_1['credit_history'].replace('no credits/all paid', 'no credits')
df_1['credit_history'] = df_1['credit_history'].replace('all paid', 'no credits')
# Reemplazar valores con mwnos de 90 en la columna 'purpose' por 'other'
df_1['purpose'] = df_1['purpose'].replace('education', 'other')
df_1['purpose'] = df_1['purpose'].replace('repairs', 'other')
df_1['purpose'] = df_1['purpose'].replace('domestic appliance', 'other')
df_1['purpose'] = df_1['purpose'].replace('retraining', 'other')
# Reemplazar valores por > 100 y mantener < 100 y 'no known savings'
df_1['savings_status'] = df_1['savings_status'].replace('100<=X<500', '>100')
df_1['savings_status'] = df_1['savings_status'].replace('500<=X<1000', '>100')
df_1['savings_status'] = df_1['savings_status'].replace('>=1000', '>100')
# eliminar la columna 'employment'
df_1.drop('employment', axis=1, inplace=True)
# Dividir la columna personal_status en dos clumnas: gender y marital_status
df_1[['gender', 'marital_status']] = df_1['personal_status'].str.split(' ', n=1, expand=True)
# Eliminar la columna 'personal_status'
df_1.drop('personal_status', axis=1, inplace=True)
# Reemplazar valores en la columna 'marital_status'
df_1['marital_status'] = df_1['marital_status'].replace('mar/wid', 'not single')
df_1['marital_status'] = df_1['marital_status'].replace('div/sep', 'not single')
df_1['marital_status'] = df_1['marital_status'].replace('div/dep/mar', 'not single')
# eliminar la columna 'other_parties'
df_1.drop('other_parties', axis=1, inplace=True)
# eliminar la columna 'other_payment_plans'
df_1.drop('other_payment_plans', axis=1, inplace=True)
# convertir la columna 'job' en una columna binaria skilled y unskilled
df_1['job']=df_1['job'].replace('unemp/unskilled non res', 'unskilled')
df_1['job']=df_1['job'].replace('unskilled resident', 'unskilled')
df_1['job']=df_1['job'].replace('high qualif/self emp/mgmt', 'skilled')
# eliminar la columna 'own_telephone'
df_1.drop('own_telephone', axis=1, inplace=True)
# eliminar la columna 'foreign_worker'
df_1.drop('foreign_worker', axis=1, inplace=True)
# convertir la columna objetivo en numerica
df_1['class']=df_1['class'].replace('good', 1)
df_1['class']=df_1['class'].replace('bad', 0)

In [26]:
df_1.head()

,duration,credit_history,purpose,credit_amount,savings_status,installment_commitment,residence_since,property_magnitude,age,housing,existing_credits,job,num_dependents,class,gender,marital_status
0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,4.0,4.0,real estate,67.0,own,2.0,skilled,1.0,1,male,single
1,48.0,existing paid,radio/tv,5951.0,<100,2.0,2.0,real estate,22.0,own,1.0,skilled,1.0,0,female,not single
2,12.0,critical/other existing credit,other,2096.0,<100,2.0,3.0,real estate,49.0,own,1.0,unskilled,2.0,1,male,single
3,42.0,existing paid,furniture/equipment,7882.0,<100,2.0,4.0,life insurance,45.0,for free,1.0,skilled,2.0,1,male,single
4,24.0,delayed previously,new car,4870.0,<100,3.0,4.0,no known property,53.0,for free,2.0,skilled,2.0,0,male,single


Introducción y antecedentes del conjunto de datos:
El conjunto de datos contiene 1000 filas y 21 columnas con 13 columnas categóricas, 7 columnas numéricas y una columna objetivo 'Class'. Se realizó un proceso de limpieza de datos y reducción de dimensionalid a través de la Ingenieria de Caracteristicas para facilitar el análisis. El objetivo está desbalanceado y se convirtió a numérico en el preprocesamiento. Algunas columnas categóricas tuvieron valores inconsistentes, mientras que otras tuvieron demasiadas categorías y se agruparon en menos categorías. Las columnas 'Checking_status', 'employment', 'other_parties' y 'own_telephone' se eliminaron porque no eran relevantes para el modelo. Los valores de las columnas que se eliminaron eran inconsistentes y en algunos casos con casi todos los valores iguales.

Las columnas 'credit_history', 'purpose', 'savings_status' y 'property_magnitude' se convirtieron en dummy. La columna 'personal_status' se renombró a 'gender' y se convirtió en dummy. La columna 'job' se convirtió en binaria. La columna 'foreign_worker' se eliminó porque no era relevante para el modelo.

Este conjunto de datos ofrece una buena oportunidad para analizar las características de los clientes y su relación con la aprobación de créditos, con la posibilidad de construir un modelo predictivo para predecir si un cliente será o no aprobado para un crédito.

Modelos de Clasificación

In [27]:
#Formato para ML y train test split
X = df_1.drop(columns = 'class')
y = df_1['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 123, stratify = y)
# Instanciar los selectores de columnas categóricas y numéricas para seleccionar las columnas adecuadas
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')
# Instanciar el escalador estándar y el codificador one hot
scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore')
# Make tuples for preprocessing the categorical and numeric columns
num_tuple = (scaler, num_selector)
cat_tuple = (ohe, cat_selector)
#Instanciación de ColumnTransformer
col_transformer = make_column_transformer(num_tuple, cat_tuple, remainder = 'passthrough')
#Observen que SOLO ENCAJAREMOS LOS DATOS DE ENTRENAMIENTO
col_transformer.fit(X_train)
#Ahora podemos transformar los conjuntos de entrenamiento y de prueba.
X_train_processed = col_transformer.transform(X_train)
X_test_processed = col_transformer.transform(X_test)
#Ver las transformaciones en dataframe
X_train_df = pd.DataFrame(X_train_processed)
X_test_df = pd.DataFrame(X_test_processed)
print(np.isnan(X_train_df).sum().sum(), 'missing values in training data')
print(np.isnan(X_test_df).sum().sum(), 'missing values in testing data')
print('\n')
print('All data in X_train_processed are', X_train_processed.dtype)
print('All data in X_test_processed are', X_test_processed.dtype)
print('\n')
print('shape of data is', X_train_df.shape)
print('\n')
X_train_df.head()

0 missing values in training data
0 missing values in testing data


All data in X_train_processed are float64
All data in X_test_processed are float64


shape of data is (750, 33)




,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,1.244061,2.684089,-0.845753,-0.765547,0.853000,0.986324,2.291288,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,0.255316,-0.680787,0.931039,-1.669735,-0.838754,0.986324,-0.436436,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,-0.733429,-0.745299,0.931039,1.042831,-1.105872,-0.710335,-0.436436,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.255316,-0.916502,0.931039,1.042831,1.654357,0.986324,-0.436436,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,-0.486243,-0.833913,0.931039,0.138642,-0.215476,0.986324,2.291288,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0


In [28]:
# Arbol de decision default y Metricas de clasificacion
dt = DecisionTreeClassifier(random_state=123)
dt.fit(X_train_df, y_train)
y_pred = dt.predict(X_test_df)
y_pred_train = dt.predict(X_train_df)
# calcular la matriz de confusión
confusion_matrix(y_test, y_pred)
# calcular la precision
precision_score(y_test, y_pred)
# calcular el recall
recall_score(y_test, y_pred)
# calcular el f1
f1_score(y_test, y_pred)
# calcular el accuracy
accuracy_score(y_test, y_pred)
# calcular el accuracy en los datos de entrenamiento
accuracy_score(y_train, y_pred_train)
#  calcular el area bajo la curva ROC
roc_auc_score(y_test, y_pred)
# imprimir todos los resultados
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print('Accuracy test: ', accuracy_score(y_test, y_pred))
print('Accuracy train: ', accuracy_score(y_train, y_pred_train))
print('ROC AUC: ', roc_auc_score(y_test, y_pred))

Confusion Matrix: 
 [[ 35  40]
 [ 50 125]]
Precision:  0.7575757575757576
Recall:  0.7142857142857143
F1:  0.7352941176470589
Accuracy test:  0.64
Accuracy train:  1.0
ROC AUC:  0.5904761904761905


In [29]:
# Buscar los mejores hiperparametros para el arbol de decision
# Definir los valores de los hiperparametros a probar
criterion = ['gini', 'entropy']
max_depth = [2, 3, 4, 5, 6, 7, 8, 9, 10]
min_samples_split = [2, 3, 4, 5, 6, 7, 8, 9, 10]
# Crear el diccionario con los valores de los hiperparametros
hyperparameters = dict(criterion=criterion, max_depth=max_depth, min_samples_split=min_samples_split)
# Instanciar el modelo
dt = DecisionTreeClassifier(random_state=123)
# Instanciar la búsqueda en rejilla
gridsearch = GridSearchCV(dt, hyperparameters, cv=5, verbose=1)
# Ajustar el modelo
best_model = gridsearch.fit(X_train_df, y_train)
# Ver los mejores hiperparametros
print('Best Criterion:', best_model.best_estimator_.get_params()['criterion'])
print('Best max_depth:', best_model.best_estimator_.get_params()['max_depth'])
print('Best min_samples_split:', best_model.best_estimator_.get_params()['min_samples_split'])
print('\n')
# Predecir con el modelo ajustado
y_pred = best_model.predict(X_test_df)
y_pred_train = best_model.predict(X_train_df)
# calcular la matriz de confusión
confusion_matrix(y_test, y_pred)
# calcular la precision
precision_score(y_test, y_pred)
# calcular el recall
recall_score(y_test, y_pred)
# calcular el f1
f1_score(y_test, y_pred)
# calcular el accuracy
accuracy_score(y_test, y_pred)
# calcular el accuracy en los datos de entrenamiento
accuracy_score(y_train, y_pred_train)
#  calcular el area bajo la curva ROC
roc_auc_score(y_test, y_pred)
# imprimir todos los resultados
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print('Accuracy test: ', accuracy_score(y_test, y_pred))
print('Accuracy train: ', accuracy_score(y_train, y_pred_train))
print('ROC AUC: ', roc_auc_score(y_test, y_pred))

Fitting 5 folds for each of 162 candidates, totalling 810 fits
Best Criterion: gini
Best max_depth: 7
Best min_samples_split: 7


Confusion Matrix: 
 [[ 25  50]
 [ 31 144]]
Precision:  0.7422680412371134
Recall:  0.8228571428571428
F1:  0.7804878048780487
Accuracy test:  0.676
Accuracy train:  0.828
ROC AUC:  0.5780952380952381


In [30]:
# Bagging Tree default y Metricas de clasificacion
bag = BaggingClassifier(random_state=123)
bag.fit(X_train_df, y_train)
y_pred = bag.predict(X_test_df)
y_pred_train = bag.predict(X_train_df)
# calcular la matriz de confusión
confusion_matrix(y_test, y_pred)
# calcular la precision
precision_score(y_test, y_pred)
# calcular el recall
recall_score(y_test, y_pred)
# calcular el f1
f1_score(y_test, y_pred)
# calcular el accuracy
accuracy_score(y_test, y_pred)
# calcular el accuracy en los datos de entrenamiento
accuracy_score(y_train, y_pred_train)
#  calcular el area bajo la curva ROC
roc_auc_score(y_test, y_pred)
# imprimir todos los resultados
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print('Accuracy test: ', accuracy_score(y_test, y_pred))
print('Accuracy train: ', accuracy_score(y_train, y_pred_train))
print('ROC AUC: ', roc_auc_score(y_test, y_pred))

Confusion Matrix: 
 [[ 32  43]
 [ 30 145]]
Precision:  0.7712765957446809
Recall:  0.8285714285714286
F1:  0.7988980716253444
Accuracy test:  0.708
Accuracy train:  0.9893333333333333
ROC AUC:  0.6276190476190476


In [31]:
# Buscar los mejores hiperparametros para el bagging tree
# Definir los valores de los hiperparametros a probar
n_estimators = [10, 20, 30, 40, 50, 60, 70, 80, 90]
max_samples = [0.1, 0.2, 0.3, 0.4, 0.5]
max_features = [0.1, 0.2, 0.3, 0.4, 0.5]
# Crear el diccionario con los valores de los hiperparametros
hyperparameters = dict(n_estimators=n_estimators, max_samples=max_samples, max_features=max_features)
# Instanciar el modelo
bag = BaggingClassifier(random_state=123)
# Instanciar la búsqueda en rejilla
gridsearch = GridSearchCV(bag, hyperparameters, cv=5, verbose=1)
# Ajustar el modelo
best_model = gridsearch.fit(X_train_df, y_train)
# Ver los mejores hiperparametros
print('Best n_estimators:', best_model.best_estimator_.get_params()['n_estimators'])
print('Best max_samples:', best_model.best_estimator_.get_params()['max_samples'])
print('Best max_features:', best_model.best_estimator_.get_params()['max_features'])
print('\n')
# Predecir con el modelo ajustado
y_pred = best_model.predict(X_test_df)
y_pred_train = best_model.predict(X_train_df)
# calcular la matriz de confusión
confusion_matrix(y_test, y_pred)
# calcular la precision
precision_score(y_test, y_pred)
# calcular el recall
recall_score(y_test, y_pred)
# calcular el f1
f1_score(y_test, y_pred)
# calcular el accuracy
accuracy_score(y_test, y_pred)
# calcular el accuracy en los datos de entrenamiento
accuracy_score(y_train, y_pred_train)
#  calcular el area bajo la curva ROC
roc_auc_score(y_test, y_pred)
# imprimir todos los resultados
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print('Accuracy test: ', accuracy_score(y_test, y_pred))
print('Accuracy train: ', accuracy_score(y_train, y_pred_train))
print('ROC AUC: ', roc_auc_score(y_test, y_pred))

Fitting 5 folds for each of 225 candidates, totalling 1125 fits
Best n_estimators: 50
Best max_samples: 0.4
Best max_features: 0.5


Confusion Matrix: 
 [[ 13  62]
 [  5 170]]
Precision:  0.7327586206896551
Recall:  0.9714285714285714
F1:  0.8353808353808354
Accuracy test:  0.732
Accuracy train:  0.9293333333333333
ROC AUC:  0.5723809523809524


In [32]:
# Random Forest default y Metricas de clasificacion
rf = RandomForestClassifier(random_state=123)
rf.fit(X_train_df, y_train)
y_pred = rf.predict(X_test_df)
y_pred_train = rf.predict(X_train_df)
# calcular la matriz de confusión
confusion_matrix(y_test, y_pred)
# calcular la precision
precision_score(y_test, y_pred)
# calcular el recall
recall_score(y_test, y_pred)
# calcular el f1
f1_score(y_test, y_pred)
# calcular el accuracy
accuracy_score(y_test, y_pred)
# calcular el accuracy en los datos de entrenamiento
accuracy_score(y_train, y_pred_train)
#  calcular el area bajo la curva ROC
roc_auc_score(y_test, y_pred)
# imprimir todos los resultados
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print('Accuracy test: ', accuracy_score(y_test, y_pred))
print('Accuracy train: ', accuracy_score(y_train, y_pred_train))
print('ROC AUC: ', roc_auc_score(y_test, y_pred))

Confusion Matrix: 
 [[ 22  53]
 [ 15 160]]
Precision:  0.7511737089201878
Recall:  0.9142857142857143
F1:  0.8247422680412371
Accuracy test:  0.728
Accuracy train:  1.0
ROC AUC:  0.6038095238095238


In [33]:
# Buscar los mejores hiperparametros para el random forest
# Definir los valores de los hiperparametros a probar
n_estimators = [10, 20, 30, 40, 50, 60, 70, 80, 90]
max_depth = [2, 3, 4, 5, 6, 7, 8, 9, 10]
max_features = [0.1, 0.2, 0.3, 0.4, 0.5]
# Crear el diccionario con los valores de los hiperparametros
hyperparameters = dict(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features)
# Instanciar el modelo
rf = RandomForestClassifier(random_state=123)
# Instanciar la búsqueda en rejilla
gridsearch = GridSearchCV(rf, hyperparameters, cv=5, verbose=1)
# Ajustar el modelo
best_model = gridsearch.fit(X_train_df, y_train)
# Ver los mejores hiperparametros
print('Best n_estimators:', best_model.best_estimator_.get_params()['n_estimators'])
print('Best max_depth:', best_model.best_estimator_.get_params()['max_depth'])
print('Best max_features:', best_model.best_estimator_.get_params()['max_features'])
print('\n')
# Predecir con el modelo ajustado
y_pred = best_model.predict(X_test_df)
y_pred_train = best_model.predict(X_train_df)
# calcular la matriz de confusión
confusion_matrix(y_test, y_pred)
# calcular la precision
precision_score(y_test, y_pred)
# calcular el recall
recall_score(y_test, y_pred)
# calcular el f1
f1_score(y_test, y_pred)
# calcular el accuracy
accuracy_score(y_test, y_pred)
# calcular el accuracy en los datos de entrenamiento
accuracy_score(y_train, y_pred_train)
#  calcular el area bajo la curva ROC
roc_auc_score(y_test, y_pred)
# imprimir todos los resultados
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print('Accuracy test: ', accuracy_score(y_test, y_pred))
print('Accuracy train: ', accuracy_score(y_train, y_pred_train))
print('ROC AUC: ', roc_auc_score(y_test, y_pred))

Fitting 5 folds for each of 405 candidates, totalling 2025 fits
Best n_estimators: 20
Best max_depth: 10
Best max_features: 0.2


Confusion Matrix: 
 [[ 20  55]
 [ 15 160]]
Precision:  0.7441860465116279
Recall:  0.9142857142857143
F1:  0.8205128205128205
Accuracy test:  0.72
Accuracy train:  0.9413333333333334
ROC AUC:  0.5904761904761905


In [34]:
# KNN default y Metricas de clasificacion
knn = KNeighborsClassifier()
knn.fit(X_train_df, y_train)
y_pred = knn.predict(X_test_df)
y_pred_train = knn.predict(X_train_df)
# calcular la matriz de confusión
confusion_matrix(y_test, y_pred)
# calcular la precision
precision_score(y_test, y_pred)
# calcular el recall
recall_score(y_test, y_pred)
# calcular el f1
f1_score(y_test, y_pred)
# calcular el accuracy
accuracy_score(y_test, y_pred)
# calcular el accuracy en los datos de entrenamiento
accuracy_score(y_train, y_pred_train)
#  calcular el area bajo la curva ROC
roc_auc_score(y_test, y_pred)
# imprimir todos los resultados
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print('Accuracy test: ', accuracy_score(y_test, y_pred))
print('Accuracy train: ', accuracy_score(y_train, y_pred_train))
print('ROC AUC: ', roc_auc_score(y_test, y_pred))

Confusion Matrix: 
 [[ 19  56]
 [ 17 158]]
Precision:  0.7383177570093458
Recall:  0.9028571428571428
F1:  0.8123393316195372
Accuracy test:  0.708
Accuracy train:  0.7973333333333333
ROC AUC:  0.5780952380952381


In [35]:
# Buscar los mejores hiperparametros para el KNN
# Definir los valores de los hiperparametros a probar
n_neighbors = [1, 2, 3, 4, 5, 6, 7, 8, 9]
weights = ['uniform', 'distance']
# Crear el diccionario con los valores de los hiperparametros
hyperparameters = dict(n_neighbors=n_neighbors, weights=weights)
# Instanciar el modelo
knn = KNeighborsClassifier()
# Instanciar la búsqueda en rejilla
gridsearch = GridSearchCV(knn, hyperparameters, cv=5, verbose=1)
# Ajustar el modelo
best_model = gridsearch.fit(X_train_df, y_train)
# Ver los mejores hiperparametros
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])
print('Best weights:', best_model.best_estimator_.get_params()['weights'])
print('\n')
# Predecir con el modelo ajustado
y_pred = best_model.predict(X_test_df)
y_pred_train = best_model.predict(X_train_df)
# calcular la matriz de confusión
confusion_matrix(y_test, y_pred)
# calcular la precision
precision_score(y_test, y_pred)
# calcular el recall
recall_score(y_test, y_pred)
# calcular el f1
f1_score(y_test, y_pred)
# calcular el accuracy
accuracy_score(y_test, y_pred)
# calcular el accuracy en los datos de entrenamiento
accuracy_score(y_train, y_pred_train)
#  calcular el area bajo la curva ROC
roc_auc_score(y_test, y_pred)
# imprimir todos los resultados
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print('Accuracy test: ', accuracy_score(y_test, y_pred))
print('Accuracy train: ', accuracy_score(y_train, y_pred_train))
print('ROC AUC: ', roc_auc_score(y_test, y_pred))

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best n_neighbors: 9
Best weights: distance


Confusion Matrix: 
 [[ 15  60]
 [  9 166]]
Precision:  0.7345132743362832
Recall:  0.9485714285714286
F1:  0.8279301745635911
Accuracy test:  0.724
Accuracy train:  1.0
ROC AUC:  0.5742857142857143


In [36]:
# Regresion Logistica default y Metricas de clasificacion
lr = LogisticRegression()
lr.fit(X_train_df, y_train)
y_pred = lr.predict(X_test_df)
y_pred_train = lr.predict(X_train_df)
# calcular la matriz de confusión
confusion_matrix(y_test, y_pred)
# calcular la precision
precision_score(y_test, y_pred)
# calcular el recall
recall_score(y_test, y_pred)
# calcular el f1
f1_score(y_test, y_pred)
# calcular el accuracy
accuracy_score(y_test, y_pred)
# calcular el accuracy en los datos de entrenamiento
accuracy_score(y_train, y_pred_train)
#  calcular el area bajo la curva ROC
roc_auc_score(y_test, y_pred)
# imprimir todos los resultados
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print('Accuracy test: ', accuracy_score(y_test, y_pred))
print('Accuracy train: ', accuracy_score(y_train, y_pred_train))
print('ROC AUC: ', roc_auc_score(y_test, y_pred))

Confusion Matrix: 
 [[ 25  50]
 [ 17 158]]
Precision:  0.7596153846153846
Recall:  0.9028571428571428
F1:  0.825065274151436
Accuracy test:  0.732
Accuracy train:  0.764
ROC AUC:  0.618095238095238


In [41]:
# Buscar los mejores hiperparametros para la Regresion Logistica
# Definir los valores de los hiperparametros a probar
penalty = ['l2']
C = np.logspace(0, 4, 10)
max_iter = [100, 500, 1000]
# Crear el diccionario con los valores de los hiperparametros
hyperparameters = dict(C=C, penalty=penalty, max_iter=max_iter)
# Instanciar el modelo
lr = LogisticRegression()
# Instanciar la búsqueda en rejilla
gridsearch = GridSearchCV(lr, hyperparameters, cv=5, verbose=1)
# Ajustar el modelo
best_model = gridsearch.fit(X_train_df, y_train)
# Ver los mejores hiperparametros
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])
print('Best max_iter:', best_model.best_estimator_.get_params()['max_iter'])
print('\n')
# Predecir con el modelo ajustado
y_pred = best_model.predict(X_test_df)
y_pred_train = best_model.predict(X_train_df)
# calcular la matriz de confusión
confusion_matrix(y_test, y_pred)
# calcular la precision
precision_score(y_test, y_pred)
# calcular el recall
recall_score(y_test, y_pred)
# calcular el f1
f1_score(y_test, y_pred)
# calcular el accuracy
accuracy_score(y_test, y_pred)
# calcular el accuracy en los datos de entrenamiento
accuracy_score(y_train, y_pred_train)
#  calcular el area bajo la curva ROC
roc_auc_score(y_test, y_pred)
# imprimir todos los resultados
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print('Accuracy test: ', accuracy_score(y_test, y_pred))
print('Accuracy train: ', accuracy_score(y_train, y_pred_train))
print('ROC AUC: ', roc_auc_score(y_test, y_pred))

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Penalty: l2
Best C: 1.0
Best max_iter: 100


Confusion Matrix: 
 [[ 25  50]
 [ 17 158]]
Precision:  0.7596153846153846
Recall:  0.9028571428571428
F1:  0.825065274151436
Accuracy test:  0.732
Accuracy train:  0.764
ROC AUC:  0.618095238095238


Modelos de Boosting

In [54]:
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier

In [50]:
# XGBoost default y Metricas de clasificacion
xgb = xgb.XGBClassifier()
xgb.fit(X_train_df, y_train)
y_pred = xgb.predict(X_test_df)
y_pred_train = xgb.predict(X_train_df)
# calcular la matriz de confusión
confusion_matrix(y_test, y_pred)
# calcular la precision
precision_score(y_test, y_pred)
# calcular el recall
recall_score(y_test, y_pred)
# calcular el f1
f1_score(y_test, y_pred)
# calcular el accuracy
accuracy_score(y_test, y_pred)
# calcular el accuracy en los datos de entrenamiento
accuracy_score(y_train, y_pred_train)
#  calcular el area bajo la curva ROC
roc_auc_score(y_test, y_pred)
# imprimir todos los resultados
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print('Accuracy test: ', accuracy_score(y_test, y_pred))
print('Accuracy train: ', accuracy_score(y_train, y_pred_train))
print('ROC AUC: ', roc_auc_score(y_test, y_pred))

Confusion Matrix: 
 [[ 31  44]
 [ 22 153]]
Precision:  0.7766497461928934
Recall:  0.8742857142857143
F1:  0.8225806451612904
Accuracy test:  0.736
Accuracy train:  1.0
ROC AUC:  0.6438095238095238


In [53]:
# Xgboost con GridSearchCV
# Definir los valores de los hiperparametros a probar
learning_rate = [0.1, 0.2, 0.3]
max_depth = [3, 4, 5]
n_estimators = [100, 200, 300]
# Crear el diccionario con los valores de los hiperparametros
hyperparameters = dict(learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators)
# Instanciar el modelo
xgb_model = XGBClassifier()
# Instanciar la búsqueda en rejilla
gridsearch = GridSearchCV(xgb_model, hyperparameters, cv=5, verbose=1)
# Ajustar el modelo
best_model = gridsearch.fit(X_train_df, y_train)
# Ver los mejores hiperparametros
print('Best learning_rate:', best_model.best_estimator_.get_params()['learning_rate'])
print('Best max_depth:', best_model.best_estimator_.get_params()['max_depth'])
print('Best n_estimators:', best_model.best_estimator_.get_params()['n_estimators'])
print('\n')
# Predecir con el modelo ajustado
y_pred = best_model.predict(X_test_df)
y_pred_train = best_model.predict(X_train_df)
# calcular la matriz de confusión
confusion_matrix(y_test, y_pred)
# calcular la precision
precision_score(y_test, y_pred)
# calcular el recall
recall_score(y_test, y_pred)
# calcular el f1
f1_score(y_test, y_pred)
# calcular el accuracy
accuracy_score(y_test, y_pred)
# calcular el accuracy en los datos de entrenamiento
accuracy_score(y_train, y_pred_train)
#  calcular el area bajo la curva ROC
roc_auc_score(y_test, y_pred)
# imprimir todos los resultados
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print('Accuracy test: ', accuracy_score(y_test, y_pred))
print('Accuracy train: ', accuracy_score(y_train, y_pred_train))
print('ROC AUC: ', roc_auc_score(y_test, y_pred))

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best learning_rate: 0.2
Best max_depth: 3
Best n_estimators: 100


Confusion Matrix: 
 [[ 25  50]
 [ 19 156]]
Precision:  0.7572815533980582
Recall:  0.8914285714285715
F1:  0.8188976377952756
Accuracy test:  0.724
Accuracy train:  0.9173333333333333
ROC AUC:  0.6123809523809525


In [55]:
# LightGBM default y Metricas de clasificacion
lgb = lgb.LGBMClassifier()
lgb.fit(X_train_df, y_train)
y_pred = lgb.predict(X_test_df)
y_pred_train = lgb.predict(X_train_df)
# calcular la matriz de confusión
confusion_matrix(y_test, y_pred)
# calcular la precision
precision_score(y_test, y_pred)
# calcular el recall
recall_score(y_test, y_pred)
# calcular el f1
f1_score(y_test, y_pred)
# calcular el accuracy
accuracy_score(y_test, y_pred)
# calcular el accuracy en los datos de entrenamiento
accuracy_score(y_train, y_pred_train)
#  calcular el area bajo la curva ROC
roc_auc_score(y_test, y_pred)
# imprimir todos los resultados
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print('Accuracy test: ', accuracy_score(y_test, y_pred))
print('Accuracy train: ', accuracy_score(y_train, y_pred_train))
print('ROC AUC: ', roc_auc_score(y_test, y_pred))

Confusion Matrix: 
 [[ 32  43]
 [ 21 154]]
Precision:  0.7817258883248731
Recall:  0.88
F1:  0.8279569892473119
Accuracy test:  0.744
Accuracy train:  1.0
ROC AUC:  0.6533333333333333


In [56]:
# LightGBM con GridSearchCV
# Definir los valores de los hiperparametros a probar
learning_rate = [0.1, 0.2, 0.3]
max_depth = [3, 4, 5]
n_estimators = [100, 200, 300]
# Crear el diccionario con los valores de los hiperparametros
hyperparameters = dict(learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators)
# Instanciar el modelo
lgb_model = LGBMClassifier()
# Instanciar la búsqueda en rejilla
gridsearch = GridSearchCV(lgb_model, hyperparameters, cv=5, verbose=1)
# Ajustar el modelo
best_model = gridsearch.fit(X_train_df, y_train)
# Ver los mejores hiperparametros
print('Best learning_rate:', best_model.best_estimator_.get_params()['learning_rate'])
print('Best max_depth:', best_model.best_estimator_.get_params()['max_depth'])
print('Best n_estimators:', best_model.best_estimator_.get_params()['n_estimators'])
print('\n')
# Predecir con el modelo ajustado
y_pred = best_model.predict(X_test_df)
y_pred_train = best_model.predict(X_train_df)
# calcular la matriz de confusión
confusion_matrix(y_test, y_pred)
# calcular la precision
precision_score(y_test, y_pred)
# calcular el recall
recall_score(y_test, y_pred)
# calcular el f1
f1_score(y_test, y_pred)
# calcular el accuracy
accuracy_score(y_test, y_pred)
# calcular el accuracy en los datos de entrenamiento
accuracy_score(y_train, y_pred_train)
#  calcular el area bajo la curva ROC
roc_auc_score(y_test, y_pred)
# imprimir todos los resultados
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print('Accuracy test: ', accuracy_score(y_test, y_pred))
print('Accuracy train: ', accuracy_score(y_train, y_pred_train))
print('ROC AUC: ', roc_auc_score(y_test, y_pred))

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best learning_rate: 0.1
Best max_depth: 3
Best n_estimators: 100


Confusion Matrix: 
 [[ 25  50]
 [ 19 156]]
Precision:  0.7572815533980582
Recall:  0.8914285714285715
F1:  0.8188976377952756
Accuracy test:  0.724
Accuracy train:  0.86
ROC AUC:  0.6123809523809525


In [57]:
# GradientBoostingClassifier default y Metricas de clasificacion
gbc = GradientBoostingClassifier()
gbc.fit(X_train_df, y_train)
y_pred = gbc.predict(X_test_df)
y_pred_train = gbc.predict(X_train_df)
# calcular la matriz de confusión
confusion_matrix(y_test, y_pred)
# calcular la precision
precision_score(y_test, y_pred)
# calcular el recall
recall_score(y_test, y_pred)
# calcular el f1
f1_score(y_test, y_pred)
# calcular el accuracy
accuracy_score(y_test, y_pred)
# calcular el accuracy en los datos de entrenamiento
accuracy_score(y_train, y_pred_train)
#  calcular el area bajo la curva ROC
roc_auc_score(y_test, y_pred)
# imprimir todos los resultados
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print('Accuracy test: ', accuracy_score(y_test, y_pred))
print('Accuracy train: ', accuracy_score(y_train, y_pred_train))
print('ROC AUC: ', roc_auc_score(y_test, y_pred))

Confusion Matrix: 
 [[ 21  54]
 [ 16 159]]
Precision:  0.7464788732394366
Recall:  0.9085714285714286
F1:  0.8195876288659794
Accuracy test:  0.72
Accuracy train:  0.888
ROC AUC:  0.5942857142857143


In [58]:
# GradientBoostingClassifier con GridSearchCV
# Definir los valores de los hiperparametros a probar
learning_rate = [0.1, 0.2, 0.3]
max_depth = [3, 4, 5]
n_estimators = [100, 200, 300]
# Crear el diccionario con los valores de los hiperparametros
hyperparameters = dict(learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators)
# Instanciar el modelo
gbc_model = GradientBoostingClassifier()
# Instanciar la búsqueda en rejilla
gridsearch = GridSearchCV(gbc_model, hyperparameters, cv=5, verbose=1)
# Ajustar el modelo
best_model = gridsearch.fit(X_train_df, y_train)
# Ver los mejores hiperparametros
print('Best learning_rate:', best_model.best_estimator_.get_params()['learning_rate'])
print('Best max_depth:', best_model.best_estimator_.get_params()['max_depth'])
print('Best n_estimators:', best_model.best_estimator_.get_params()['n_estimators'])
print('\n')
# Predecir con el modelo ajustado
y_pred = best_model.predict(X_test_df)
y_pred_train = best_model.predict(X_train_df)
# calcular la matriz de confusión
confusion_matrix(y_test, y_pred)
# calcular la precision
precision_score(y_test, y_pred)
# calcular el recall
recall_score(y_test, y_pred)
# calcular el f1
f1_score(y_test, y_pred)
# calcular el accuracy
accuracy_score(y_test, y_pred)
# calcular el accuracy en los datos de entrenamiento
accuracy_score(y_train, y_pred_train)
#  calcular el area bajo la curva ROC
roc_auc_score(y_test, y_pred)
# imprimir todos los resultados
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print('Accuracy test: ', accuracy_score(y_test, y_pred))
print('Accuracy train: ', accuracy_score(y_train, y_pred_train))
print('ROC AUC: ', roc_auc_score(y_test, y_pred))

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best learning_rate: 0.1
Best max_depth: 3
Best n_estimators: 100


Confusion Matrix: 
 [[ 21  54]
 [ 16 159]]
Precision:  0.7464788732394366
Recall:  0.9085714285714286
F1:  0.8195876288659794
Accuracy test:  0.72
Accuracy train:  0.888
ROC AUC:  0.5942857142857143


Arbol de Decision
Confusion Matrix: 
 [[ 25  50]
 [ 31 144]]
Precision:  0.7422680412371134
Recall:  0.8228571428571428
F1:  0.7804878048780487
Accuracy test:  0.676
Accuracy train:  0.828
ROC AUC:  0.5780952380952381

Bagging Classifier
Confusion Matrix: 
 [[ 13  62]
 [  5 170]]
Precision:  0.7327586206896551
Recall:  0.9714285714285714
F1:  0.8353808353808354
Accuracy test:  0.732
Accuracy train:  0.9293333333333333
ROC AUC:  0.5723809523809524

Random Forest
Confusion Matrix: 
 [[ 20  55]
 [ 15 160]]
Precision:  0.7441860465116279
Recall:  0.9142857142857143
F1:  0.8205128205128205
Accuracy test:  0.72
Accuracy train:  0.9413333333333334
ROC AUC:  0.5904761904761905

KNN Classifier
Confusion Matrix: 
 [[ 15  60]
 [  9 166]]
Precision:  0.7345132743362832
Recall:  0.9485714285714286
F1:  0.8279301745635911
Accuracy test:  0.724
Accuracy train:  1.0
ROC AUC:  0.5742857142857143

Regresion Logistica
Confusion Matrix: 
 [[ 25  50]
 [ 17 158]]
Precision:  0.7596153846153846
Recall:  0.9028571428571428
F1:  0.825065274151436
Accuracy test:  0.732
Accuracy train:  0.764
ROC AUC:  0.618095238095238

XGBoost
Confusion Matrix: 
 [[ 25  50]
 [ 19 156]]
Precision:  0.7572815533980582
Recall:  0.8914285714285715
F1:  0.8188976377952756
Accuracy test:  0.724
Accuracy train:  0.9173333333333333
ROC AUC:  0.6123809523809525

LightGBM
Confusion Matrix: 
 [[ 25  50]
 [ 19 156]]
Precision:  0.7572815533980582
Recall:  0.8914285714285715
F1:  0.8188976377952756
Accuracy test:  0.724
Accuracy train:  0.86
ROC AUC:  0.6123809523809525

Gradient Boosting
Confusion Matrix: 
 [[ 21  54]
 [ 16 159]]
Precision:  0.7464788732394366
Recall:  0.9085714285714286
F1:  0.8195876288659794
Accuracy test:  0.72
Accuracy train:  0.888
ROC AUC:  0.5942857142857143

Para seleccionar el mejor modelo, debemos considerar varias métricas de evaluación. En general, buscamos un equilibrio entre la precisión, el recall, el F1-score y el ROC AUC, y buscamos la mayor precisión posible junto con un buen equilibrio entre recall y F1-score.

En este caso, el modelo que muestra el mejor rendimiento general y un equilibrio aceptable entre las métricas es la Regresión Logística. Tiene una precisión de 0.760, un recall de 0.903 y un F1-score de 0.825. Además, el ROC AUC de 0.618 indica un rendimiento razonable para la curva ROC.

La Regresión Logística también tiene un buen equilibrio entre el rendimiento en los conjuntos de entrenamiento y prueba, lo que sugiere una capacidad de generalización adecuada. Sin embargo, es importante tener en cuenta que este análisis se basa únicamente en los resultados proporcionados y no considera otros factores, como la complejidad del modelo y los requisitos específicos del problema.

En resumen, el modelo elegido para predecir si un cliente es bueno o malo para tomar un crédito es la Regresión Logística